# Overview
---

The goal of this workshop is to gain familiarity with the high performance computing (HPC) resources provided by Compute Canada. The workshop will cover aspects of logging in, transfering data, initializing environments, requesting resources (CPU / GPU), and relinquishing jobs when you are done using them. To motivate the use of these resources, a deep learning model written in Python will be trained on both CPU and GPU, and the relative performances will be evaluated. 

A large amount of information in these iPython notebooks is summarized from the [SHARCNet knowledge base](https://www.sharcnet.ca/help/index.php/Knowledge_Base), and this [webinar](https://www.youtube.com/watch?v=IiAbxPZ3BHo&feature=youtu.be)


## Workshop Requirements
---

To follow along with this workshop, you will need the following:
* Compute Canada account
* (WINDOWS) [Putty](https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html) for connecting to SHARCNet
* (WINDOWS) [Psftp](https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html) for transfering files between SHARCNet
* (OPTIONAL, WINDOWS) [xming](http://www.straightrunning.com/XmingNotes/) for viewing graphical windows 

## Compute Canada / SHARCNet Nodes
---

Compute Canada houses many different HPC clusters, which put simply are simply large pools of physical CPU / GPU systems configured in a special way. A list of Compute Canada nodes can be found [here](https://www.computecanada.ca/research-portal/accessing-resources/available-resources/). Each cluster is configured differently, and their configuration can be found on their corresponding wiki pages. For example:
* [Cedar](https://docs.computecanada.ca/wiki/Cedar): cedar.computecanada.ca 
* [Graham](https://docs.computecanada.ca/wiki/Graham): graham.computecanada.ca
* [Niagara](niagara.computecanada.ca): niagra.computecanada.ca

__SHARCNet__ stands for _Shared Hierarchical Academic Research Computing Network_, and is a partner of Compute Canada. It was established in 2000, and involves 18 Universities and Colleges across Southern, Central, and Northern Ontario. The main office is at the University of Western Ontario. SHARCNet and Compute Canada are free to use for academic researchers.

Note the path at the end of each bullet. This is the end-point you will need to connect to in order to log in and access these resources.

# Logging In

For more information on establishing a connection (including logging in), see [here](https://docs.computecanada.ca/wiki/SSH)

In order to log in to Compute Canada _or_ SHARCNet, you need to establish a connection using the secure shell protocol (SSH). On Linux this protocol is built into the kernel and accessed via the terminal, while on windows an external program may be required. 


## 1.1. Connecting from Linux
---

If connecting from linux, entering the following from the command line: 
 
```ssh graham.computecanada.ca``` 

and you will be prompted to input your username and password. Alternatively, you can also initialize the connection with your username by adding it as a prefix

```ssh <username>@graham.computecanada.ca```

By pre-pending your username, you will only need to enter your password on the following screen. 

## 1.2. Connecting from Windows
---

While recent versions of Windows have included support for many Linux utilities (for example, [Windows Subsystem for Linux](https://docs.microsoft.com/en-us/windows/wsl/install-win10)), you may find yourself needed a seperate program to log in if the ```ssh``` command is not available from the command line. 

Download the program [Putty](https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html), and enter &lt;cluster_name&gt;.computecanada.ca in the "Host Name (or IP address) field. You shouldn't need to change any other sections, but note that capabilities such as port-forwarding and X11 forwarding are available if needed.

<img src="images/graham_sharcnet.PNG" width="400">


# 2. Major Feature: Login Nodes vs. Development Nodes
---

When you connect to Compute Canada or SHARCNet, you connect to a _login_ node that serves as a sort-of centralized hub. Login nodes are the only nodes that have internet access, and are useful for tasks such as:

* Submitting jobs to the scheduler
* Transfering data to / from the SHARCNet ecosystem
* Initializing code environments
* Coding / processing very small projects

A different kind of node known as a _development node_ is the main workhorses designed to handle heavy code processing.


## 2.1. Interactive Nodes vs. Non-Interactive Nodes
---

When submitting a job to the scheduler, you give it constraints such as the maximum running time (i.e. amount of you need a CPUs / GPUs for), and the amount of memory your program will need to run. Given these constraints, the job scheduler will find an appropriate time for your program to run on _non-interactive nodes_. If your program is found to have bugs in it (and it crashes during its alloted execution time), your remaining time will be forfeit and you will be tasked with fixing the code and submitting it for execution again. 

Interactive nodes on the other hand, can be used to quickly prototype code on a particular resource. You can request either CPUs or GPUs for a small amount of time, do some interactive programming, and when you're done relinquish the node and submit the job to a non-interactive node. Thus, it's extremely common to use both interactive and non-interactive nodes when developing models. An example pipeline is to: log-in --> transfer code & data --> request an interactive node to verify program runs --> submit a job. 

# 3. Major Feature: The Job Scheduler
---

Information from [here](https://www.sharcnet.ca/help/index.php/SLURM) and webinar [here](https://www.youtube.com/watch?v=dfw027j284Q)

If you are unfamiliar with HPC, Compute Canada / SHARCNet may be very different then what you are used to. In this setting, an entity known as a _scheduler_ controls what programs are allowed to run when. This helps to ensure fairness in how the physical resources are used by all researchers. The more resources you request, the longer you will likely need to wait until you are granted usage rights. This scheduler is called __SLURM__, and stands for: Simple Linux Utility for Resource Management. 

An overview of the job scheduling policies can be found [here](https://docs.computecanada.ca/wiki/Job_scheduling_policies).

#  4. Major Feature / Consideration: Data Storage Policies
---

There are a number of different resources available for storing data within the Compute Canada environment. The medium you choose to use to host things such as code or datasets can be based on the attributes in the following table (Taken from [here](https://docs.computecanada.ca/wiki/Storage_and_file_management)) 


|   Filesystem  |           Default Quota          |            Lustre-based?           |  Backed up? |                  Purged?                |  Available by Default? |  Mounted on Compute Nodes? |
|:-------------:|:--------------------------------:|:----------------------------------:|:-----------:|:---------------------------------------:|:----------------------:|----------------------------|
| Home Space    | 50 GB and 500K files per user[1] | Yes for Cedar, No for Graham (NFS) | Yes         | No                                      | Yes                    | Yes                        |
| Scratch Space | 20 TB and 1M files per user[2]   | Yes                                | No          | Files older than 60 days are purged.[3] | Yes                    | Yes                        |
| Project Space | 1 TB and 500k files per group[4] | Yes                                | Yes         | No                                      | Yes                    | Yes                        |

There are a few important points to address: 

* The __/home__ folder is accessed by all users, and as such, should not be used for any extremely large datasets, or programs that frequently do a lot of IO. The more IO a program does, the more latency you introduce to the system, which causes network congestion and unresponsiveness. As /home is backed up, large datasets that do not frequently change do not need to be backed up, and can be stored on other partitions such as /scratch.
* The __/scratch__ is fast access, and often the preferred directory for storing single-user data. However, files on /scratch will only exist for 60 days before they get purged. This means that if your program writes any files to the /scratch file system and you want to keep these files, you should move them once done. 

# 5. Consideration: File Transfer
---

There are many different ways to transfer data between your compute and the SHARCNet resources. If you are not comfortable using the command line to transfer data, SHARCNet encourages the use of software known as [Globus](https://www.sharcnet.ca/help/index.php/Globus). This is a portal that operates in a manner similar to "drag and drop" that most users should be familiar with. 

There are also other methods for transferring data such as:

* (LINUX) [scp](https://linux.die.net/man/1/scp) which stands for _secure copy_
* (LINUX) [rsync](https://linux.die.net/man/1/rsync) which is an efficient tool for copying files
* (WINDOWS) [psftp](https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html) 

These will be discussed further in the workshop.